In [25]:
import numpy as np
import os
import PIL
import PIL.Image
import glob
from skimage import io
from skimage.transform import resize
import random
import matplotlib.pyplot as plt
import pandas as pd
import subprocess as sp

In [26]:
# Set a random seed for the train/val/test split
random.seed(170)

In [27]:
input_dir = '../data/raw/'
out_dir = '../data/reservoirs_10band/'

In [28]:
sp.call(['mkdir', '-p', out_dir + 'img_dir/train'])
sp.call(['mkdir', '-p', out_dir + 'img_dir/val'])
sp.call(['mkdir', '-p', out_dir + 'img_dir/test'])

sp.call(['mkdir', '-p', out_dir + 'ann_dir/train'])
sp.call(['mkdir', '-p', out_dir + 'ann_dir/val'])
sp.call(['mkdir', '-p', out_dir + 'ann_dir/test'])


0

In [29]:
val_frac = 0.2
test_frac = 0.2

## First round of loading data:
 1. Save mins and maxes of every band for every image
 
(Can be commented out once we've calculated it once and have saved "all_imgs_bands_min_max_v1.npy")

In [30]:
# # Get mask image names and base image patterns
# mask_images = glob.glob('{}*mask.png'.format(input_dir))
# mask_images.sort()
# image_patterns = [mi.replace('mask.png', '') for mi in mask_images]
    
# # To calculate minimum and maximums of each band for scaling
# band_mins = []
# band_maxes = []
                 
# # Load each image and store minimum and maximum values of each band
# for image_base in image_patterns:
#     stacked_ar = np.concatenate([
#         io.imread('{}og.tif'.format(image_base)),
#         io.imread('{}s1_v2_og.tif'.format(image_base)),
#         io.imread('{}s2_20m_og.tif'.format(image_base)),

#     ], axis=2)
    
#     img_min = np.min(stacked_ar, axis=(0,1))
#     img_max = np.max(stacked_ar, axis=(0,1))
    
    
#     band_mins += [img_min]
#     band_maxes += [img_max]

# all_mins = np.stack(band_mins)
# all_maxes = np.stack(band_maxes)
# bands_min_max_all_imgs = np.stack([all_mins, all_maxes], axis=0)
# np.save('../data/all_imgs_bands_min_max_v1.npy', bands_min_max_all_imgs)

## Second round of loading data:
1. Split into train, val, test
2. Rescale bands using the previously calculated min and max
3. Calc NDs
4. Select bands for feeding into CNN
5. Calc mean and std in running fashion for those bands, save


In [31]:
# For each band, calculate the minumum from the minumum values
# And the 80th percentile of the maximum values
# If we use the max of maximums, the values get too stretched out, but you might need to tweak this
bands_min_max_all_imgs = np.load('../data/all_imgs_bands_min_max_v1.npy')
bands_min_max = np.array([np.min(bands_min_max_all_imgs[0], axis=0),
                          np.percentile(bands_min_max_all_imgs[1], 80, axis=0)])

In [32]:
def normalized_diff(ar1, ar2):
    """Returns normalized difference of two arrays."""
    return np.nan_to_num(((ar1 - ar2) / (ar1 + ar2)),0)
    
    
def calc_nd(img, band1, band2):
    """Add band containing NDWI."""

    nd = normalized_diff(img[:,:,band1].astype('float64'), 
                         img[:,:,band2].astype('float64'))
    
    # Rescale to uint8
    nd = np.round(255.*(nd - (-1))/(1 - (-1)))
    if nd.max()>255:
        print(nd.max())
        print('Error: overflow')
   
    return nd.astype(np.uint8)

def calc_all_nds(img):
    
    nd_list =[]
    
    # Add  Gao NDWI
    nd_list += [calc_nd(img, 3, 11)]
    # Add  MNDWI
    nd_list += [calc_nd(img, 1, 11)]
    # Add McFeeters NDWI band
    nd_list += [calc_nd(img, 1, 3)]
    # Add NDVI band
    nd_list += [calc_nd(img, 3, 2)]

    return np.stack(nd_list, axis=2)


In [38]:
def rescale_to_minmax_uint8(img, bands_min_max):
    img = np.where(img > bands_min_max[1], bands_min_max[1], img)
    img  = (255. * (img.astype('float64') - bands_min_max[0]) / (bands_min_max[1] - bands_min_max[0]))
    img = np.round(img)
    if img.max()>255:
        print(img.max())
        print('Error: overflow')
    return img.astype(np.uint8)

def select_bands_save_tif(fp_base, out_path, band_selection, bands_min_max, crop_size, resample_size,
                          calc_mean_std=False):
    ar = np.concatenate([
        io.imread('{}og.tif'.format(fp_base)),
        io.imread('{}s1_v2_og.tif'.format(fp_base)),
        io.imread('{}s2_20m_og.tif'.format(fp_base))
    ], axis=2)

    ar = rescale_to_minmax_uint8(ar, bands_min_max)
    
    nds = calc_all_nds(ar)
    
    ar = np.concatenate([ar, nds], axis=2)[:, :, band_selection]
    
    if ar.shape[:-1] != crop_size:
        ar = crop(ar, crop_size)
    if ar.shape[:-1] != resample_size:
        ar = resize(ar, resample_size, preserve_range=True).astype(np.uint8)
    io.imsave(out_path, ar)
    if calc_mean_std:
        return ar.reshape((-1, len(band_selection)))
    
def crop(img, out_size):
    crop_size = int((img.shape[0] - out_size[0])/2)
    img = img
    return img[crop_size:(-crop_size), crop_size:(-crop_size)]

def create_nband_tifs(img_list, output_dir, band_selection = [0, 1, 2, 3, 4, 5, 12, 13, 14, 15], crop_size=(640,640),
                      resample_size=(640, 640),
                      calc_mean_std=False):
    if calc_mean_std:
        n = 0
        mean = np.zeros(len(band_selection))
        sums =  np.zeros(len(band_selection))
        M2 =  np.zeros(len(band_selection))

    for fp_base in img_list:
        # out_path is a little tricky, need to remove _ at end and add in .png
        out_path = fp_base.replace(input_dir, output_dir)[:-1] + '.tif'
        if calc_mean_std:
            vals = select_bands_save_tif(fp_base, out_path, band_selection, 
                                         bands_min_max, crop_size, resample_size, 
                                         calc_mean_std=calc_mean_std)
            n += vals.shape[0]
            vals = vals
            sums += np.sum(vals, axis=0)
            delta = vals - mean
            mean += np.sum(delta/n, axis=0)
            M2 += np.sum(delta*(vals - mean), axis=0)
        else:
            select_bands_save_tif(fp_base, out_path, band_selection, bands_min_max, crop_size, resample_size)
            
    if calc_mean_std:
        return sums/n, np.sqrt(M2 / (n - 1))
    
        
def save_mask_pngs(ann_list, output_dir, resample_size=(500, 500)):
    for fp_base in ann_list:
        fp = '{}mask.png'.format(fp_base)
        ar = io.imread(fp)
        ar[ar>0] = 1
        
        # Sometimes the pngs have 3 dims. We only want 2
        if len(ar.shape) > 2:
            ar = ar[:,:,0]
            
        
        # Replace with all-zero mask f only 1 positive pixel
        # This was a quirk in the old labelbox style
        ar_sum = ar.sum()
        if ar_sum == 1:
            ar[:] = 0
            
        # Resize
        if ar.shape != resample_size:
            ar = np.round(resize(ar, resample_size, preserve_range=True)).astype(np.uint8)

        if np.sum(np.logical_and(ar!=1, ar!=0)) > 0:
            print(fp)
            print(np.unique(ar))
            raise ValueError('Mask has non-0 and/or non-1 values')
            
        # Save
        out_path = fp.replace(input_dir, output_dir).replace('_mask.png', '.png')
        io.imsave(out_path, ar)

In [39]:
def split_train_test(img_patterns, test_frac, val_frac):
    """Split data into train, test, val (or just train, or just train and test

    Returns:
        train_indices, val_indices, test_indices tuple
    """
    total_ims = len(img_patterns)
    if test_frac != 0:

        train_count = round(total_ims * (1 - test_frac - val_frac))
        train_indices = random.sample(range(total_ims), train_count)
        test_val_indices = np.delete(np.array(range(total_ims)), train_indices)

        test_count = round(total_ims * test_frac)
        test_indices = random.sample(list(test_val_indices), test_count)


        if val_frac != 0:
            val_indices = np.delete(np.array(range(total_ims)),
                                    np.append(train_indices, test_indices))

            return train_indices, val_indices, test_indices
        else: 
            return train_indices, test_indices
    else:
        return np.arange(total_ims)

# Get train, test, and val lists
def list_and_split_imgs(input_dir):
    """Get image paths list and split into train, test, val"""
    # First get list of images
    mask_images = glob.glob('{}*mask.png'.format(input_dir))
    mask_images.sort()
    image_patterns = np.array([mi.replace('mask.png', '') for mi in mask_images])
    
    # Split
    train_indices, val_indices, test_indices = split_train_test(image_patterns, test_frac=test_frac, val_frac=val_frac)
    train_basename_list = image_patterns[train_indices]
    val_basename_list = image_patterns[val_indices]
    test_basename_list = image_patterns[test_indices]
    
    return train_basename_list, val_basename_list, test_basename_list

In [40]:
# Run the split
train_basename_list, val_basename_list, test_basename_list = list_and_split_imgs(input_dir)

In [41]:
# Save preprocessed images
means_std = create_nband_tifs(train_basename_list, output_dir = '{}/img_dir/train/'.format(out_dir),
                              calc_mean_std = True)
create_nband_tifs(test_basename_list, output_dir = '{}/img_dir/test/'.format(out_dir), calc_mean_std = False)
create_nband_tifs(val_basename_list, output_dir = '{}/img_dir/val/'.format(out_dir), calc_mean_std = False)

(640, 640, 12)
../data/raw/im_0000116480-0000000000_34_og.tif
(640, 640, 12)
../data/raw/im_0000046592-0000139776_4_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000116480_74_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000163072_73_og.tif
(640, 640, 12)
../data/raw/im_0000163072-0000093184_62_og.tif
(640, 640, 12)
../data/raw/im_0000093184-0000046592_75_og.tif
(640, 640, 12)
../data/raw/im_0000069888-0000069888_56_og.tif
(640, 640, 12)
../data/raw/im_0000116480-0000163072_42_og.tif
(640, 640, 12)
../data/raw/im_0000023296-0000163072_27_og.tif
(640, 640, 12)
../data/raw/im_0000209664-0000139776_25_og.tif
(640, 640, 12)
../data/raw/im_0000023296-0000186368_51_og.tif
(640, 640, 12)
../data/raw/im_0000116480-0000023296_15_og.tif
(640, 640, 12)
../data/raw/im_0000232960-0000116480_26_og.tif
(640, 640, 12)
../data/raw/im_0000093184-0000093184_64_og.tif
(640, 640, 12)
../data/raw/im_0000116480-0000046592_42_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000023296_2_og.tif
(640, 640,

(640, 640, 12)
../data/raw/im_0000046592-0000163072_54_og.tif
(640, 640, 12)
../data/raw/im_0000209664-0000116480_74_og.tif
(640, 640, 12)
../data/raw/im_0000186368-0000116480_34_og.tif
(640, 640, 12)
../data/raw/im_0000093184-0000186368_49_og.tif
(640, 640, 12)
../data/raw/im_0000116480-0000093184_17_og.tif
(640, 640, 12)
../data/raw/im_0000186368-0000139776_77_og.tif
(640, 640, 12)
../data/raw/im_0000163072-0000093184_31_og.tif
(640, 640, 12)
../data/raw/im_0000232960-0000139776_13_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000116480_54_og.tif
(640, 640, 12)
../data/raw/im_0000209664-0000093184_34_og.tif
(640, 640, 12)
../data/raw/im_0000232960-0000139776_66_og.tif
(640, 640, 12)
../data/raw/im_0000046592-0000023296_36_og.tif
(640, 640, 12)
../data/raw/im_0000069888-0000069888_8_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000046592_41_og.tif
(640, 640, 12)
../data/raw/im_0000000000-0000186368_59_og.tif
(640, 640, 12)
../data/raw/im_0000186368-0000093184_12_og.tif
(640, 640

(640, 640, 12)
../data/raw/im_0000186368-0000163072_16_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000093184_17_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000209664_54_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000023296_74_og.tif
(640, 640, 12)
../data/raw/im_0000232960-0000139776_51_og.tif
(640, 640, 12)
../data/raw/im_0000186368-0000116480_61_og.tif
(640, 640, 12)
../data/raw/im_0000023296-0000186368_31_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000186368_29_og.tif
(640, 640, 12)
../data/raw/im_0000232960-0000139776_56_og.tif
(640, 640, 12)
../data/raw/im_0000186368-0000046592_69_og.tif
(640, 640, 12)
../data/raw/im_0000116480-0000069888_47_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000093184_40_og.tif
(640, 640, 12)
../data/raw/im_0000069888-0000139776_25_og.tif
(640, 640, 12)
../data/raw/im_0000186368-0000139776_31_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000046592_0_og.tif
(640, 640, 12)
../data/raw/im_0000069888-0000116480_77_og.tif
(640, 640

/tmp/ipykernel_993620/1699935328.py:3: RuntimeWarning: invalid value encountered in true_divide
  return np.nan_to_num(((ar1 - ar2) / (ar1 + ar2)),0)


(640, 640, 12)
../data/raw/im_0000093184-0000186368_33_og.tif
(640, 640, 12)
../data/raw/im_0000209664-0000139776_79_og.tif
(640, 640, 12)
../data/raw/im_0000069888-0000046592_31_og.tif
(640, 640, 12)
../data/raw/im_0000093184-0000069888_77_og.tif
(640, 640, 12)
../data/raw/im_0000163072-0000116480_39_og.tif
(640, 640, 12)
../data/raw/im_0000116480-0000069888_67_og.tif
(640, 640, 12)
../data/raw/im_0000069888-0000163072_40_og.tif
(640, 640, 12)
../data/raw/im_0000093184-0000046592_22_og.tif
(640, 640, 12)
../data/raw/im_0000093184-0000023296_35_og.tif
(640, 640, 12)
../data/raw/im_0000093184-0000023296_33_og.tif
(640, 640, 12)
../data/raw/im_0000186368-0000069888_58_og.tif
(640, 640, 12)
../data/raw/im_0000116480-0000163072_0_og.tif
(640, 640, 12)
../data/raw/im_0000186368-0000116480_69_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000046592_23_og.tif
(640, 640, 12)
../data/raw/im_0000093184-0000093184_40_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000000000_12_og.tif
(640, 640

(640, 640, 12)
../data/raw/im_0000093184-0000069888_5_og.tif
(640, 640, 12)
../data/raw/im_0000186368-0000116480_23_og.tif
(640, 640, 12)
../data/raw/im_0000186368-0000163072_13_og.tif
(640, 640, 12)
../data/raw/im_0000093184-0000093184_18_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000186368_27_og.tif
(640, 640, 12)
../data/raw/im_0000186368-0000139776_18_og.tif
(640, 640, 12)
../data/raw/im_0000116480-0000046592_73_og.tif
(640, 640, 12)
../data/raw/im_0000232960-0000139776_9_og.tif
(640, 640, 12)
../data/raw/im_0000069888-0000023296_51_og.tif
(640, 640, 12)
../data/raw/im_0000163072-0000186368_8_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000093184_24_og.tif
(640, 640, 12)
../data/raw/im_0000209664-0000116480_9_og.tif
(640, 640, 12)
../data/raw/im_0000023296-0000209664_21_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000093184_69_og.tif
(640, 640, 12)
../data/raw/im_0000069888-0000069888_68_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000069888_63_og.tif
(640, 640, 1

(640, 640, 12)
../data/raw/im_0000116480-0000186368_10_og.tif
(640, 640, 12)
../data/raw/im_0000116480-0000209664_33_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000023296_24_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000023296_4_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000046592_74_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000046592_75_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000093184_15_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000093184_21_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000093184_32_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000093184_46_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000093184_50_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000116480_33_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000116480_36_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000116480_4_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000139776_17_og.tif
(640, 640, 12)
../data/raw/im_0000139776-0000163072_23_og.tif
(640, 640,

In [42]:
# Save preprocessed 
save_mask_pngs(train_basename_list, output_dir = '{}/ann_dir/train/'.format(out_dir))
save_mask_pngs(test_basename_list, output_dir = '{}/ann_dir/test/'.format(out_dir))
save_mask_pngs(val_basename_list, output_dir = '{}/ann_dir/val/'.format(out_dir))

/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000116480-0000000000_34.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000046592-0000139776_4.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000139776-0000116480_74.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000139776-0000163072_73.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000163072-0000093184_62.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000093184-0000046592_75

/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000093184-0000069888_32.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000139776-0000186368_38.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000163072-0000023296_8.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000069888-0000023296_59.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000116480-0000023296_64.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000163072-0000093184_79

/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000046592-0000186368_76.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000046592-0000116480_14.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000000000-0000186368_5.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000069888-0000139776_57.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000139776-0000069888_47.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000116480-0000023296_34

/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000093184-0000139776_5.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000093184-0000069888_55.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000186368-0000163072_64.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000093184-0000139776_71.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000116480-0000023296_69.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000046592-0000163072_29

/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000023296-0000163072_65.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000116480-0000023296_62.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000116480-0000046592_48.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000186368-0000093184_36.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000139776-0000186368_52.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/train/im_0000139776-0000069888_6

...

/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/test/im_0000186368-0000139776_7.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/test/im_0000139776-0000000000_73.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/test/im_0000163072-0000116480_57.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/test/im_0000069888-0000000000_22.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/test/im_0000116480-0000093184_59.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/test/im_0000069888-0000046592_34.png i

/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/test/im_0000209664-0000139776_55.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/test/im_0000046592-0000163072_14.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/test/im_0000000000-0000209664_1.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/test/im_0000093184-0000023296_53.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/test/im_0000139776-0000046592_40.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/test/im_0000093184-0000046592_59.png i

/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000046592-0000163072_42.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000046592-0000163072_48.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000046592-0000163072_58.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000046592-0000186368_23.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000046592-0000209664_4.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000046592-0000209664_79.png is a lo

/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000116480-0000163072_9.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000116480-0000186368_10.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000116480-0000209664_33.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000139776-0000023296_24.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000139776-0000023296_4.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000139776-0000046592_74.png is a low

/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000186368-0000116480_56.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000186368-0000116480_8.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000186368-0000139776_49.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000186368-0000139776_70.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000186368-0000139776_9.png is a low contrast image
  io.imsave(out_path, ar)
/tmp/ipykernel_993620/1333191448.py:95: UserWarning: ../data/reservoirs_10band//ann_dir/val/im_0000186368-0000163072_65.png is a low

In [43]:
np.save('../data/mean_std_v1.npy', np.vstack(means_std))